# Abadi Federated for sign:

In [ ]:
# Baseline Model on the Sonar Dataset
import keras
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout,PReLU, LeakyReLU, Embedding, LSTM, Convolution1D
from keras.initializers import lecun_uniform, Constant
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
import sys
# Local toolset
import predictive
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Lambda
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding, Flatten, Dense
from keras.initializers import lecun_uniform, Constant, glorot_normal, he_normal, glorot_uniform, he_uniform
from keras.callbacks import ModelCheckpoint
from keras import regularizers
import gc
from keras import backend as K_s
import json
from sklearn.preprocessing import MinMaxScaler
import secrets, copy
from numpy import linalg as LA
#from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.utils import class_weight


import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [ ]:
def sign(w_t,w_t_1):
    for i in range(0,len(w_t)):
        w_t_1[i]=np.sign(w_t_1[i]-w_t[i])
        w_t_1[i][w_t_1[i]==0]=np.random.choice([-1,1], w_t_1[i][w_t_1[i]==0].shape, replace=True) 
    return w_t_1

In [ ]:
def Metrics(list_hosp,X_val,Y_val,X_val_0,Y_val_0,X_val_1,Y_val_1):
    
    metrics=[]
    
    model= load_model("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5")
    

    tmp=model.evaluate(X_val_0, Y_val_0, batch_size=len(Y_val_0), verbose=0)
    
    acc_0=tmp[1]
    loss_0=tmp[0]


    tmp=model.evaluate(X_val_1, Y_val_1, batch_size=len(Y_val_1), verbose=0)

    acc_1=tmp[1]
    loss_1=tmp[0]


    tmp=(loss_0+loss_1)/2
    
    metrics.append(tmp)
    
    tmp=(acc_0+acc_1)/2
    
    metrics.append(tmp)
    
    
    pred=model.predict(X_val,batch_size=len(Y_val))
    
    pred = pred.reshape(Y_val.shape)
    
    stats = predictive.stats.evaluate(predictive.stats.DataKind.PANDAS, pred, Y_val, .5, sys.stdout)
    
    metrics.append(stats)
        
    return metrics 

In [ ]:
def create_model():
    K_s.clear_session()
    gc.collect()
    
    input_dim=24428
    
    model = Sequential()
    model.add(Dense(200,input_dim=input_dim, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    
    return model

In [ ]:
# coté client:
def ClientUpdate(k, w_t, sigma):
    
    X_train= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/X_train_"+str(k)+".npy")
    Y_train= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/Y_train_"+str(k)+".npy")
    
    
    w_t_tmp=copy.deepcopy(w_t)
    
    model= create_model()
    
    model.set_weights(w_t_tmp)
    
    class_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train),Y_train)

    class_weights = dict(enumerate(class_weights))
    
    model.fit(X_train, Y_train, epochs=5,batch_size=len(Y_train),verbose=0,class_weight= class_weights)
    
    del X_train, Y_train
    gc.collect()
    
    return(sign(w_t,model.get_weights()))

In [ ]:
X_val= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/X_test.npy")
Y_val= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/Y_test.npy")


X_val_0=copy.deepcopy(X_val[Y_val[:]==0])
Y_val_0=copy.deepcopy(Y_val[Y_val[:]==0])


X_val_1=copy.deepcopy(X_val[Y_val[:]==1])
Y_val_1=copy.deepcopy(Y_val[Y_val[:]==1])


sigma_list=[0] #[1.08,0.81,0.63]


for sigma in sigma_list:


    list_hosp=np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Hospitals_id.npy")

    
    
    print("list_hosp_length:",len(list_hosp))

    model = create_model()

    model.save("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5")
    gc.collect()


    round=0
    res_suivie=[]


    K_s.clear_session()
    del model
    gc.collect()


    res_suivie.append([Metrics(list_hosp,X_val,Y_val,X_val_0,Y_val_0,X_val_1,Y_val_1)])

    with open("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Results/Sign_"+str(round)+"_sigma_"+str(sigma)+".json", 'w') as f:
        json.dump(res_suivie, f)


    # coté serveur

    N= 314 # Nbr clients
    C= 3/314# ratio of clients 0.1
    K= max(C*N,1)

    for round in range(1,301):
        w_k_t_1={}
        print(round)
        S_t=np.random.choice(list_hosp, int(np.round(K)), replace=False) # select a set of hospitals

        w_t= load_model("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5").get_weights()

        cpt=0
        for k in S_t:
            cpt+=1
            print(cpt)
            w_k_t_1[k]= ClientUpdate(k,w_t, sigma)
            K_s.clear_session()

        vote=[]
        ctrl=0
        for k in w_k_t_1:

            if(ctrl==0):
                for indice_arg in range(0,len(w_k_t_1.get(k))):
                    vote.append(np.add(0,w_k_t_1.get(k)[indice_arg]))
                ctrl+=1

            else:
                for indice_arg in range(0,len(w_k_t_1.get(k))):
                    vote[indice_arg]=np.add(vote[indice_arg],w_k_t_1.get(k)[indice_arg])

        del w_k_t_1


        for i in range(0,len(vote)):
            vote[i]=np.sign(vote[i])
            w_t[i]=np.add(w_t[i],np.multiply(vote[i],0.001))
            

        model = create_model()


        model.set_weights(w_t)

        del w_t

        model.save("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5")



        if((round)%1==0):
            res_suivie.append([Metrics(list_hosp,X_val,Y_val,X_val_0,Y_val_0,X_val_1,Y_val_1)])
            with open("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Results/Sign_"+str(round)+"_sigma_"+str(sigma)+".json", 'w') as f:
                 json.dump(res_suivie, f)

        del model
        K_s.clear_session()
        gc.collect()